In [2]:
# My OpenAI Key
import os
os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_KEY')

In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [139]:
## Using sdsc-data-llama collection from MongoDB to populate LLamaIndex
from llama_index.storage.index_store import MongoIndexStore
from llama_index import StorageContext, VectorStoreIndex, SimpleMongoReader, load_index_from_storage
from IPython.display import Markdown, display

host = "localhost"
port = 27017
db_name = "mydatabase"
collection_name = 'sdsc-data-llama'

index_store = MongoIndexStore.from_host_and_port(
    host = host,
    port = port,
    db_name = db_name,
    namespace = f'{db_name}.{collection_name}'
)

# create storage context
storage_context = StorageContext.from_defaults(index_store=index_store)

query_dict = {}
field_names = ["news", "title", "publishdate"]
reader = SimpleMongoReader(host, port)
documents = reader.load_data(db_name, collection_name, field_names, query_dict=query_dict)

In [5]:
print(len(documents))

2951


In [61]:
import time

batch = 50
index = VectorStoreIndex.from_documents(documents[:batch])

for i in range(batch, len(documents), batch):
    print(i)
    if batch + i > len(documents):
        index = index.from_documents(documents[i:])
    else:
        index = index.from_documents(documents[i:batch + i])
    time.sleep(10)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 72038 tokens
> [build_index_from_nodes] Total embedding token usage: 72038 tokens
50
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 64188 tokens
> [build_index_from_nodes] Total embedding token usage: 64188 tokens
100
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 85787 tokens
> [build_index_from_node

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 52338 tokens
> [build_index_from_nodes] Total embedding token usage: 52338 tokens
1200
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 53074 tokens
> [build_index_from_nodes] Total embedding token usage: 53074 tokens
1250
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 64100 tokens
> [build_index_from_nodes] Total embedding token usage: 64100 tokens
1300
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [bui

2350
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 83447 tokens
> [build_index_from_nodes] Total embedding token usage: 83447 tokens
2400
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 53221 tokens
> [build_index_from_nodes] Total embedding token usage: 53221 tokens
2450
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 77240 tokens
> [build_index_f

TypeError: 'method' object is not subscriptable

In [26]:
#index = VectorStoreIndex.from_documents(documents)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 72038 tokens
> [build_index_from_nodes] Total embedding token usage: 72038 tokens


In [137]:
index.storage_context.persist(persist_dir = './storage')

In [141]:
index.set_index_id('model1')

In [142]:
index.index_id

'model1'

In [62]:
query_engine = index.as_query_engine()

In [63]:
# West Virginia
response = query_engine.query(
    """
    Using the documents, answer this question:
    Who are the major party nominess in the West Virginia Senate Race in 2018?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 37 tokens
> [retrieve] Total embedding token usage: 37 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1717 tokens
> [get_response] Total LLM token usage: 1717 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>The major party nominees in the West Virginia Senate Race in 2018 were Joe Manchin (Democrat) and Patrick Morrisey (Republican).</b>

In [80]:
# West Virginia
response = query_engine.query(
    """
    Who are the major party nominess in the West Virginia Senate Race in 2018?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 25 tokens
> [retrieve] Total embedding token usage: 25 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 858 tokens
> [get_response] Total LLM token usage: 858 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>The major party nominees in the West Virginia Senate Race in 2018 are Joe Manchin (Democrat) and Patrick Morrisey (Republican).</b>

In [64]:
# West Virginia
response = query_engine.query(
    """
    Using the documents, answer this question:
    Who is the winner of the West Virginia Senate Race in 2018?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 34 tokens
> [retrieve] Total embedding token usage: 34 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1690 tokens
> [get_response] Total LLM token usage: 1690 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>Joe Manchin.</b>

In [81]:
# West Virginia
response = query_engine.query(
    """
    Who is the winner of the West Virginia Senate Race in 2018?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 22 tokens
> [retrieve] Total embedding token usage: 22 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 857 tokens
> [get_response] Total LLM token usage: 857 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>The winner of the West Virginia Senate Race in 2018 was Joe Manchin.</b>

In [92]:
# West Virginia
response = query_engine.query(
    """
    Using the documents, answer this question:
    How did Joe Manchin in 2018 perform in his senate reelection compare to Hillary Clinton's
    presidential comapaign in 2016? Can you provide specific information on how they both performed?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 60 tokens
> [retrieve] Total embedding token usage: 60 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1767 tokens
> [get_response] Total LLM token usage: 1767 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>Joe Manchin won his 2018 Senate reelection in West Virginia, while Hillary Clinton lost the state in her 2016 presidential campaign. Manchin won the state by a margin of 3.6%, while Clinton lost by a margin of 42.2%. Manchin was able to win the state by appealing to rural and small-town voters, while Clinton struggled with the same demographic. Manchin defended the coal industry, while Clinton's policies ran afoul of rural and small-town voters who were more culturally conservative than urban dwellers.</b>

In [93]:
# West Virginia
response = query_engine.query(
    """
    How did Joe Manchin in 2018 perform in his senate reelection compare to Hillary Clinton's
    presidential comapaign in 2016? Can you provide specific information on how they both performed?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 48 tokens
> [retrieve] Total embedding token usage: 48 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 924 tokens
> [get_response] Total LLM token usage: 924 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>Joe Manchin's net rating fell by roughly 18 percentage points by the end of 2018, while Hillary Clinton lost the 2016 presidential election to Donald Trump. Specifically, Joe Manchin won his reelection in 2018, while Hillary Clinton lost the popular vote to Donald Trump by 2.1%.</b>

In [82]:
# West Virginia
response = query_engine.query(
    """
    Using the documents, answer this question:
    What were major campaign issues in the West Virginia Senate Race.
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 33 tokens
> [retrieve] Total embedding token usage: 33 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1700 tokens
> [get_response] Total LLM token usage: 1700 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>The major campaign issues in the West Virginia Senate Race were Joe Manchin's defense of the coal industry, Sherrod Brown's opposition to much of U.S. trade policy, and Jon Tester's emphasis on his rancher credentials.</b>

In [83]:
# West Virginia
response = query_engine.query(
    """
    What were major campaign issues in the West Virginia Senate Race.
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 21 tokens
> [retrieve] Total embedding token usage: 21 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1662 tokens
> [get_response] Total LLM token usage: 1662 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>The major campaign issues in the West Virginia Senate Race were likely coal industry, U.S. trade policy, and gun measures. Senator Joe Manchin defended the coal industry, opposed much of U.S. trade policy, and was criticized by Hillary Clinton for some Senate votes against gun measures.</b>

In [87]:
# West Virginia
response = query_engine.query(
    """
    Using the documents, answer this question:
    How did Joe Manchin perform among rural voters?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 31 tokens
> [retrieve] Total embedding token usage: 31 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1677 tokens
> [get_response] Total LLM token usage: 1677 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>Joe Manchin, the Senator from West Virginia, defended the coal industry, which likely resonated with rural voters. The AP VoteCast survey found that 56 percent of rural and small-town residents voted for Republican House candidates, compared to 41 percent for Democrats. This suggests that Manchin likely performed well among rural voters.</b>

In [88]:
# West Virginia
response = query_engine.query(
    """
    How did Joe Manchin perform among rural voters?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 19 tokens
> [retrieve] Total embedding token usage: 19 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1660 tokens
> [get_response] Total LLM token usage: 1660 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>Joe Manchin, the Senator from West Virginia, likely performed well among rural voters. He is known for defending the coal industry, which is an important industry in rural areas. Additionally, he is known for opposing much of U.S. trade policy, which is also popular among rural voters.</b>

In [67]:
# Montana
response = query_engine.query(
    """
    Using the documents, answer this question:
    Who are the major party nominess in the Montana Senate Race in 2018?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 36 tokens
> [retrieve] Total embedding token usage: 36 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 581 tokens
> [get_response] Total LLM token usage: 581 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>The major party nominees in the Montana Senate Race in 2018 are Jon Tester (Democrat) and Matt Rosendale (Republican).</b>

In [89]:
# Montana
response = query_engine.query(
    """
    Who are the major party nominess in the Montana Senate Race in 2018?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 24 tokens
> [retrieve] Total embedding token usage: 24 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 855 tokens
> [get_response] Total LLM token usage: 855 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>The major party nominees in the Montana Senate Race in 2018 are Jon Tester (Democrat) and Matt Rosendale (Republican).</b>

In [90]:
# Montana
response = query_engine.query(
    """
    Using the documents, answer this question:
    Who is the winner of the Montana Senate Race in 2018?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 33 tokens
> [retrieve] Total embedding token usage: 33 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 868 tokens
> [get_response] Total LLM token usage: 868 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>Sen. Jon Tester of Montana won the Montana Senate Race in 2018.</b>

In [91]:
# Montana
response = query_engine.query(
    """
    Who is the winner of the Montana Senate Race in 2018?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 21 tokens
> [retrieve] Total embedding token usage: 21 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 844 tokens
> [get_response] Total LLM token usage: 844 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>The winner of the Montana Senate Race in 2018 is Jon Tester, the Democratic incumbent.</b>

In [101]:
# Montana
response = query_engine.query(
    """
    Using the documents, answer this question:
    How did Jon Tester perform in his 2018 senate reelection compared to other 2018 senate races?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 39 tokens
> [retrieve] Total embedding token usage: 39 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 622 tokens
> [get_response] Total LLM token usage: 622 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>Based on the information provided, it is not possible to answer this question. The documents provide information about Jon Tester's performance in the 2016 senate race, as well as his performance in the 2018 race so far. However, there is no information provided about how Jon Tester's performance compared to other 2018 senate races.</b>

In [102]:
# Montana
response = query_engine.query(
    """
    How did Jon Tester perform in his 2018 senate reelection compared to other 2018 senate races?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 27 tokens
> [retrieve] Total embedding token usage: 27 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 634 tokens
> [get_response] Total LLM token usage: 634 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>Jon Tester's 2018 senate reelection was one of the most competitive of the 26 Democratic incumbents up for reelection in 2018. His net rating fell by roughly 18 percentage points by the end of the year, the most of any of the Democratic incumbents. He received an unexpected assist from an old ally, Chuck Hagel, and his independent streak could help him with independent voters, strong Democratic turnout, and some Republican defections in November.</b>

In [99]:
# Montana
response = query_engine.query(
    """
    Using the documents, answer this question:
    What were major campaign issues in the Montana Senate Race.
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 32 tokens
> [retrieve] Total embedding token usage: 32 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1226 tokens
> [get_response] Total LLM token usage: 1226 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>Major campaign issues in the Montana Senate Race included veterans' matters, Jon Tester's independent streak, and Chuck Hagel's support for Tester. Additionally, Republican candidates Matt Rosendale and Troy Downing criticized Tester for his role in releasing what they called unsubstantiated claims against Jackson.</b>

In [100]:
# Montana
response = query_engine.query(
    """
    What were major campaign issues in the Montana Senate Race.
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 20 tokens
> [retrieve] Total embedding token usage: 20 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1190 tokens
> [get_response] Total LLM token usage: 1190 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>Major campaign issues in the Montana Senate Race included veterans' issues, Jon Tester's independent streak, and the Republican Party's response to President Trump's outbursts.</b>

In [106]:
# Montana
response = query_engine.query(
    """
    Using the documents, answer this question:
    How did Jon Tester perform among rural voters in his 2018 reelection race?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 36 tokens
> [retrieve] Total embedding token usage: 36 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1652 tokens
> [get_response] Total LLM token usage: 1652 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>Jon Tester won his 2018 reelection race, suggesting that he performed well among rural voters. He played up his rancher credentials, which likely resonated with rural voters.</b>

In [105]:
# Montana
response = query_engine.query(
    """
    How did Jon Tester perform among rural voters in his 2018 reelection race?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 24 tokens
> [retrieve] Total embedding token usage: 24 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 868 tokens
> [get_response] Total LLM token usage: 868 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>Jon Tester's net rating fell by roughly 18 percentage points among rural voters by the end of the 2018 reelection race.</b>

In [68]:
# Utah
response = query_engine.query(
    """
    Using the documents, answer this question:
    Who are the major party nominess in the Utah Senate Race in 2018?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 36 tokens
> [retrieve] Total embedding token usage: 36 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 861 tokens
> [get_response] Total LLM token usage: 861 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>The major party nominees in the Utah Senate Race in 2018 are Republican Mitt Romney and Democrat Jenny Wilson.</b>

In [107]:
# Utah
response = query_engine.query(
    """
    Who are the major party nominess in the Utah Senate Race in 2018?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 24 tokens
> [retrieve] Total embedding token usage: 24 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 849 tokens
> [get_response] Total LLM token usage: 849 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>The major party nominees in the Utah Senate Race in 2018 are Republican Mitt Romney and Democrat Jenny Wilson.</b>

In [108]:
# Utah
response = query_engine.query(
    """
    Using the documents, answer this question:
    Who is the winner of the Utah Senate Race in 2018?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 33 tokens
> [retrieve] Total embedding token usage: 33 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1469 tokens
> [get_response] Total LLM token usage: 1469 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>The winner of the Utah Senate Race in 2018 is Republican Mitt Romney.</b>

In [109]:
# Utah
response = query_engine.query(
    """
    Who is the winner of the Utah Senate Race in 2018?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 21 tokens
> [retrieve] Total embedding token usage: 21 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 840 tokens
> [get_response] Total LLM token usage: 840 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>The winner of the Utah Senate Race in 2018 is Republican Mitt Romney.</b>

In [126]:
# Utah
response = query_engine.query(
    """
    Using the documents, answer this question:
    Why was there a swing from democrats to republicans in Utah from 2016 to 2018?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 39 tokens
> [retrieve] Total embedding token usage: 39 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1473 tokens
> [get_response] Total LLM token usage: 1473 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>The documents do not provide enough information to answer this question.</b>

In [125]:
# Utah
response = query_engine.query(
    """
    Why was there a swing from democrats to republicans in Utah from 2016 to 2018?
    """
)
display(Markdown(f"<b>{response}</b>"))

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 27 tokens
> [retrieve] Total embedding token usage: 27 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1538 tokens
> [get_response] Total LLM token usage: 1538 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


<b>In Utah, the Republican advantage was driven by a surge in turnout among rural and small-town whites. In 2016, they made up 28 percent of the electorate and voted for Republican House candidates by a margin of 57-41. In 2018, that share rose to 30 percent and they voted 63-35 for Republicans. This suggests that the enthusiasm among rural and small-town whites for Republican candidates was higher in 2018 than it was in 2016.</b>